## Here is an example of the retrieval-based (search-based) chatbot.

In [1]:
import pandas as pd
from nlpia.data.loaders import get_data

C:\Users\voldo\AppData\Local\Programs\Python\Python39\lib\site-packages\pugnlp\constants.py:136: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  [datetime.datetime, pd.datetime, pd.Timestamp])
C:\Users\voldo\AppData\Local\Programs\Python\Python39\lib\site-packages\pugnlp\constants.py:158: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  MIN_TIMESTAMP = pd.Timestamp(pd.datetime(1677, 9, 22, 0, 12, 44), tz='utc')
C:\Users\voldo\AppData\Local\Programs\Python\Python39\lib\site-packages\pugnlp\tutil.py:100: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  np = pd.np
C:\Users\voldo\AppData\Local\Programs\Python\Python39\lib\site-packages\pugnlp\util.py:80: FutureWarning: The pandas.np module is deprecated and wil

In [2]:
df = get_data("ubuntu_dialog")

INFO:nlpia.loaders:Downloading ubuntu_dialog
DEBUG:nlpia.futil:regex pattern = ^[.]?([^.]*)\.([^.]{1,10})*\.300d\.zip$, string=ubuntu_dialog_1500k.csv.gz
DEBUG:nlpia.futil:regex pattern = ^[.]?([^.]*)\.([^.]{1,10})*\.27b\.zip$, string=ubuntu_dialog_1500k.csv.gz
DEBUG:nlpia.futil:regex pattern = ^[.]?([^.]*)\.([^.]{1,10})*\.42b\.zip$, string=ubuntu_dialog_1500k.csv.gz
DEBUG:nlpia.futil:regex pattern = ^[.]?([^.]*)\.([^.]{1,10})*\.6b\.zip$, string=ubuntu_dialog_1500k.csv.gz
DEBUG:nlpia.futil:regex pattern = ^[.]?([^.]*)\.([^.]{1,10})*\.bin\.gz$, string=ubuntu_dialog_1500k.csv.gz
DEBUG:nlpia.futil:regex pattern = ^[.]?([^.]*)\.([^.]{1,10})*\.tgz$, string=ubuntu_dialog_1500k.csv.gz
INFO:nlpia.loaders:expanded+normalized file path: C:\Users\voldo\AppData\Local\Programs\Python\Python39\lib\site-packages\nlpia\bigdata\ubuntu_dialog_1500k.csv.gz
INFO:nlpia.loaders:requesting URL: https://www.dropbox.com/s/krvi79fbsryytc2/ubuntu_dialog_1500k.csv.gz?dl=1
INFO:nlpia.loaders:remote_size: 296098788

In [3]:
df.head()

,Context,Utterance
0,i think we could import the old comments via r...,basically each xfree86 upload will NOT force u...
1,I'm not suggesting all - only the ones you mod...,oh? oops. __eou__
2,afternoon all __eou__ not entirely related to ...,we'll have a BOF about this __eou__ so you're ...
3,interesting __eou__ grub-install worked with /...,i fully endorse this suggestion </quimby> __eo...
4,and because Python gives Mark a woody __eou__ ...,(i thought someone was going to make a joke ab...


In [4]:
import re


def split_turns(s, splitter=re.compile("__eot__")):
    for utterance in splitter.split(s):
        utterance = utterance.replace("__eou__", "\n")
        utterance = utterance.replace("__eot__", "").strip()
        if len(utterance):
            yield utterance

In [6]:
for i, record in df.head(3).iterrows():
    statement = list(split_turns(record.Context))[-1]
    reply = list(split_turns(record.Utterance))[-1]
    print("--------------------------------")
    print(f"Statement: {statement}\n")
    print(f"Reply: {reply}")

--------------------------------
Statement: I would prefer to avoid it at this stage.  this is something that has gone into XSF svn, I assume?

Reply: basically each xfree86 upload will NOT force users to upgrade 100Mb of fonts for nothing 
 no something i did in my spare time.
--------------------------------
Statement: ok, it sounds like you're agreeing with me, then 
 though rather than "the ones we modify", my idea is "the ones we need to merge"

Reply: oh? oops.
--------------------------------
Statement: should g2 in ubuntu do the magic dont-focus-window tricks? 
 join the gang, get an x-series thinkpad 
 sj has hung on my box, again. 
 what is monday mornings discussion actually about?

Reply: we'll have a BOF about this 
 so you're coming tomorrow ?


In [9]:
from tqdm import tqdm


def preprocess_ubuntu_corpus(df_in):
    """
    Split all strings in df_in.Context and df_in.Utterance on __eot__ (turn) markers
    :param df_in: pandas.DataFrame of Context and Utterance columns
    :return:
    """
    statements = []
    replies = []
    for inx, record in tqdm(df.iterrows()):
        turns = list(split_turns(record.Context))
        statement = turns[-1] if len(turns) else "\n"
        statements.append(statement)
        turns = list(split_turns(record.Utterance))
        reply = turns[-1] if len(turns) else "\n"
        replies.append(reply)
    df_in["statement"] = statements
    df_in["reply"] = replies
    return df_in

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer


df = preprocess_ubuntu_corpus(df)
tfidf = TfidfVectorizer(min_df=8, max_df=0.3, max_features=50_000)

1499744it [01:01, 24353.03it/s]


TypeError: fit() missing 1 required positional argument: 'raw_documents'

In [11]:
tfidf.fit(df.statement)

TfidfVectorizer(max_df=0.3, max_features=50000, min_df=8)

In [12]:
X = tfidf.transform(df.statement)

NameError: name 'pd' is not defined

In [13]:
import pandas as pd

X = pd.DataFrame(X.todense(), columns=tfidf.get_feature_names())

MemoryError: Unable to allocate 290. GiB for an array with shape (1499744, 25991) and data type float64

In [16]:
x = tfidf.transform(["This is an example statement that we want to retrieve the best reply for."])
cosine_similarities = x.dot(X.T)
reply = df.loc[cosine_similarities.argmax()]

In [17]:
print(reply)

Context      Why don't all linux system support ONE packagi...
Utterance                         I think google knows __eou__
statement    i'd like to know the difference first, before ...
reply                                     I think google knows
Name: 84839, dtype: object
